In [1]:
import argparse
from Bio import Entrez

import pandas as pd
from datetime import date
from datetime import datetime
import numpy as np
import re
from metapub import FindIt
from metapub.convert import pmid2doi
from metapub.convert import doi2pmid
from metapub import PubMedFetcher

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

2022-03-08 17:01:48 rstudio metapub.config[44768] WARNING NCBI_API_KEY was not set.


## Setting the output path

In [13]:
# determining the current year, month and day
today = date.today()
date_str = today.strftime("%Y_%m_%d")

# determining current hour and minute
now = datetime.now()
time_str = now.strftime("%H_%M")

# setting the output filename
output = "GEO_Query.{}_{}".format(date_str, time_str)

## Query the NCBI database

In [3]:
# define a search query/filters
filters = "hichip[All Fields] AND (\"Homo sapiens\"[Organism] AND \"published last year\"[Filter]) AND \"gse\"[Filter]"

# set a dummy email 
Entrez.email = "zjiang@lji.org"

# query the NCBI database 
search_result = Entrez.esearch(db="gds", retmax=10000, term=filters)
result = Entrez.read(search_result)

# creating a regex to extract pubmed IDs
PubMedIds_pattern = re.compile('[0123456789]+')

In [4]:
# parse the results into a dataframe 
dictionary_lst = []
for Id in result["IdList"]:
    handle = Entrez.esummary(db="gds", id=Id, retmode="xml") # get summary of this entry on GEO datasets
    entry = Entrez.parse(handle)
    for column in entry:
        dictionary_lst.append(column)
df = pd.DataFrame(dictionary_lst)

In [5]:
df

,Item,Id,Accession,GDS,title,summary,GPL,GSE,taxon,entryType,...,ExtRelations,n_samples,SeriesTitle,PlatformTitle,PlatformTaxa,SamplesTaxa,PubMedIds,Projects,FTPLink,GEO2R
0,[],200180198,GSE180198,,EWSR1-ATF1 dependent 3D connectivity regulates...,This SuperSeries is composed of the SubSeries ...,20301,180198,Homo sapiens,GSE,...,[],147,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE180nn...,no
1,[],200180194,GSE180194,,EWSR1-ATF1 dependent 3D connectivity regulates...,Chimeric proteins resulting from chromosomal t...,20301,180194,Homo sapiens,GSE,...,"[{'RelationType': 'SRA', 'TargetObject': 'SRP3...",4,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE180nn...,no
2,[],200173872,GSE173872,,EBF1 nuclear repositioning instructs chromatin...,This SuperSeries is composed of the SubSeries ...,24676;18573,173872,Homo sapiens,GSE,...,[],233,,,,,"[IntegerElement(35182476, attributes={})]",[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE173nn...,yes
3,[],200173871,GSE173871,,EBF1 nuclear repositioning instructs chromatin...,Purpose: To investigate the mechanisms of 3D g...,18573,173871,Homo sapiens,GSE,...,"[{'RelationType': 'SRA', 'TargetObject': 'SRP3...",4,,,,,"[IntegerElement(35182476, attributes={})]",[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE173nn...,yes
4,[],200173843,GSE173843,,EBF1 nuclear repositioning instructs chromatin...,Purpose: To investigate the mechanisms of 3D g...,18573,173843,Homo sapiens,GSE,...,"[{'RelationType': 'SRA', 'TargetObject': 'SRP3...",2,,,,,"[IntegerElement(35182476, attributes={})]",[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE173nn...,yes
5,[],200186055,GSE186055,,Cohesin protein STAG2 regulates expression of ...,The cohesin complex participates in the organi...,16791,186055,Homo sapiens,GSE,...,"[{'RelationType': 'SRA', 'TargetObject': 'SRP3...",2,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE186nn...,yes
6,[],200156773,GSE156773,,Cohesin protein STAG2 regulates expression of ...,This SuperSeries is composed of the SubSeries ...,16791,156773,Homo sapiens,GSE,...,[],32,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE156nn...,no
7,[],200156772,GSE156772,,Cohesin protein STAG2 regulates expression of ...,The cohesin complex participates in the organi...,16791,156772,Homo sapiens,GSE,...,"[{'RelationType': 'SRA', 'TargetObject': 'SRP2...",4,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE156nn...,no
8,[],200156771,GSE156771,,Cohesin protein STAG2 regulates expression of ...,The cohesin complex participates in the organi...,16791,156771,Homo sapiens,GSE,...,"[{'RelationType': 'SRA', 'TargetObject': 'SRP2...",20,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE156nn...,no
9,[],200156770,GSE156770,,Cohesin protein STAG2 regulates expression of ...,The cohesin complex participates in the organi...,16791,156770,Homo sapiens,GSE,...,"[{'RelationType': 'SRA', 'TargetObject': 'SRP2...",6,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE156nn...,no


## Cleaning the Paper Names and Converting from PMID to DOI

In [6]:
dictionary_lst = []
for Id in result["IdList"]:
    
    handle = Entrez.esummary(db="gds", id=Id, retmode="xml")
    entry = Entrez.parse(handle)
    
    for column in entry:
        paper_title = column['title']
        new_paper_title = ''
        last_word = re.split('\s|[.]', paper_title)[-1]
        last_two_words = re.split('\s|[.]', paper_title)[-2:]
        
        # remove ".[HiChIP]", ". [HiChIP]", or "(HiChIP)" in paper titles using steps below
        if ('[' in last_word and ']' in last_word) or \
                ('(' in last_word and ')' in last_word):
            new_paper_title = re.split('\s|[.]', paper_title)[0:-1]
            new_paper_title = ' '.join(new_paper_title)
            new_paper_title = new_paper_title.strip()
            if new_paper_title.endswith('.'): # remove the last period in paper title
                new_paper_title = new_paper_title[:-1]
                
        # remove ".[Hi ChIP]", ". [Hi ChIP]", or "[Bead Array]" in paper titles using steps below
        elif ('[' in last_two_words[0] and ']' in last_two_words[1]) or \
                    ('(' in last_two_words[0] and ')' in last_two_words[1]):
            new_paper_title = re.split('\s|[.]', paper_title)[0:-2]
            new_paper_title = ' '.join(new_paper_title)
            new_paper_title = new_paper_title.strip()
            if new_paper_title.endswith('.'): # remove the last period in paper title
                new_paper_title = new_paper_title[:-1]
                
        elif paper_title.endswith('.'):
            new_paper_title = paper_title[:-1] 
            
        else:
            new_paper_title = paper_title
            
        new_paper_title = new_paper_title.replace("\xa0", " ") # remove no-break space
        column['title'] = new_paper_title
        
        # convert PMID to DOI, some have no PMID so empty string
        string = str(column['PubMedIds']); pdf_url = ""
        PubMedIds = PubMedIds_pattern.findall(string)
        
        if len(PubMedIds) > 0:
            DOI = pmid2doi(PubMedIds[0])
            column['PubMedIds'] = DOI
        else: column['PubMedIds'] = ""
        dictionary_lst.append(column)
df = pd.DataFrame(dictionary_lst)

MetaPubError: Invalid ID "34908129" (rejected by Eutils); please check the number and try again.

## Reformat the Columns for Google Sheet Compatibility

In [ ]:
df = pd.DataFrame(dictionary_lst)

# drop extra columns
drop_cols = ['Item', 'Id', 'GDS', 'GPL', 'GSE', 'entryType', 'ptechType',
             'valType', 'SSInfo', 'subsetInfo', 'suppFile', 'Relations', 'ExtRelations',
             'n_samples', 'SeriesTitle', 'PlatformTitle', 'PlatformTaxa', 'SamplesTaxa',
             'Projects', 'FTPLink', 'GEO2R']
df.drop(drop_cols, inplace=True, axis=1)

# rename columns
rename_cols = {"Accession":"GEO / Data link", "PubMedIds":"DOI", "title":"Paper Title",
               "taxon":"Organism", "gdsType":"Any other information", "PDAT":"Year",
               "Samples":"Other matched data"}
df.rename(columns=rename_cols, inplace=True)

# extract just the year
df["Year"] = df["Year"].str[:4]

# add index for merging
df['index'] = np.arange(len(df))

# fill na with empty values
df.fillna("",inplace=True)

# adding missing columns with empty values
df["Journal"] = ""
df["Authors"] = ""
df["Tissue/Cell Line"] = ""
df["Presenter"] = ""
df["Potential HiChIP"] = ""

# reorder the columns
reorder = ["index", "Paper Title", "DOI", "Journal", "Authors", "Year",
           "GEO / Data link", "Any other information", "Organism",
           "Tissue/Cell Line", "Potential HiChIP", "Other matched data",
           "Presenter"]
df = df[reorder]

## Assigning Potential HiChIP Samples

Filter out samples with no HiChIP mention and assigned Yes or Maybe to all others 

In [ ]:
col = df["Other matched data"]
lst = []
index = 0
for row in col:
    
    temp = []
    state = False
    
    # Check for the term HiChIP in the title
    for ele in row: 
        if ("HiChIP".casefold() in ele["Title"].casefold()) or \
                ("Hi-ChIP".casefold() in ele["Title"].casefold()):
            state = True
    
    # keep samples with HiChIP and mark "Yes"
    if state == True: 
            for ele in row:
                if ("HiChIP".casefold() in ele["Title"].casefold()) or \
                        ("Hi-ChIP".casefold() in ele["Title"].casefold()):
                    temp.append(ele["Accession"] + ": " + ele["Title"])
            temp_str = "\n".join(temp)
            df.at[index, "Potential HiChIP"] = "Yes"

    # otherwise, keep all GSM samples and mark "Maybe"
    else: 
        for ele in row:
            temp.append(ele["Accession"] + ": " + ele["Title"])
        temp_str = "\n".join(temp)
        df.at[index,"Potential HiChIP"]="Maybe"
    lst.append(temp_str)
    index += 1
df["Other matched data"] = list(lst)

## Merge Rows with the Same Paper Title

In [ ]:
aggregation_functions = {'Paper Title':'first', 'DOI':'max', 'Journal':'first', 'Authors':'first', 'Year':'first', 'GEO / Data link':lambda x: '\n'.join(x),
       'Any other information':lambda x: '\n'.join(x), 'Organism':'first', 'Tissue/Cell Line':'first',
       'Potential HiChIP':'first', 'Other matched data':lambda x: '\n'.join(x), 'Presenter':'first'}
df_grouped = df.groupby(df['Paper Title']).aggregate(aggregation_functions)
df_grouped['Date Added'] = date
df_grouped

## Fetch Journal Name and First Author of Each Paper

In [ ]:
jounrnal_lst = []
authors_lst = []
fetch = PubMedFetcher()
for DOI in df_grouped['DOI']:
    try:
        PMID = doi2pmid(DOI)
        article = fetch.article_by_pmid(PMID)
        jounrnal_lst.append(article.journal)
        authors_lst.append(article.authors[0].split()[0] + " et al.")
    except:
        jounrnal_lst.append("")
        authors_lst.append("")
df_grouped['Journal'] = jounrnal_lst
df_grouped['Authors'] = authors_lst

In [ ]:
df_grouped

## Save the final output file

In [ ]:
df_grouped.to_excel(output+".xlsx", index=False)